<a href="https://colab.research.google.com/github/activeloopai/examples/blob/main/colabs/Training_an_Image_Classification_Model_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Training an Image Classification Model in PyTorch***



#### The primary objective for Hub is to enable users to manage their data more easily so they can train better ML models. This tutorial shows you how to train a simple image classification model while streaming data from a Hub dataset stored in the cloud.

## Install Hub

In [ ]:
from IPython.display import clear_output
!pip3 install hub
clear_output()

In [ ]:
# IMPORTANT - Please restart your Colab runtime after installing Hub!
# This is a Colab-specific issue that prevents PIL from working properly.
import os
os.kill(os.getpid(), 9)

## Data Preprocessing


The first step is to select a dataset for training. This tutorial uses the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset that has already been converted into hub format. It is a simple image classification dataset that categorizes images by clothing type (trouser, shirt, etc.)

In [ ]:
import hub
from PIL import Image
import numpy as np
import os, time
import torch
from torchvision import datasets, transforms, models

ds_train = hub.load('hub://activeloop/fashion-mnist-train')
ds_test = hub.load('hub://activeloop/fashion-mnist-test')

In [ ]:
Image.fromarray(ds_train.images[0].numpy()).resize((100,100))

In [ ]:
print(ds_train.labels.info.class_names[str(ds_train.labels[0].numpy()[0])])

The next step is to define a transformation function that will process the data and convert it into a format that can be passed into a deep learning model. The syntax for the transformation function is that the input parameter is a sample from a Hub dataset in dictionary syntax, and the return value is a dictionary containing the data that the training loop uses to train the model. In this particular example, `torchvision.transforms` is used as a part of the transformation pipeline that performs operations such as normalization and image augmentation (rotation).

In [ ]:
def transform(sample_in):
    return {'images': tform(sample_in['images']), 'labels': sample_in['labels']}

tform = transforms.Compose([
    transforms.ToPILImage(), # Must convert to PIL image for subsequent operations to run
    transforms.RandomRotation(20), # Image augmentation
    transforms.ToTensor(), # Must convert to pytorch tensor for subsequent operations to run
    transforms.Normalize([0.5], [0.5]),
])

**Note:** Don't worry if the above syntax is a bit confusing 😵! We're currently improving it.

You are now ready to create a pytorch dataloader that connects the Hub dataset to the PyTorch model. This can be done using the provided method `ds.pytorch()` , which automatically applies the user-defined transformation function, takes care of random shuffling (if desired), and converts hub data to PyTorch tensors. The `num_workers` parameter can be used to parallelize data preprocessing, which is critical for ensuring that preprocessing does not bottleneck the overall training workflow.

In [ ]:
batch_size = 32

train_loader = ds_train.pytorch(num_workers = 2, shuffle = True, transform = transform, batch_size = batch_size)
test_loader = ds_test.pytorch(num_workers = 2, transform = transform, batch_size = batch_size)

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

## Model Definition

This tutorial uses a pre-trained [ResNet18](https://pytorch.org/hub/pytorch_vision_resnet/) neural network from the torchvision.models module, converted to a single-channel network for grayscale images.

In [ ]:
# Simple model can be trained on a CPU
device = torch.device("cpu")

net = models.resnet18(pretrained=True)
# Convert model to grayscale
net.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Update the fully connected layer based on the number of classes in the dataset
net.fc = torch.nn.Linear(net.fc.in_features, len(ds_train.labels.info.class_names))

net.to(device)

# Specity the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.1)

## Training the Model

Helper functions for training and testing the model are defined. Note that the dictionary that is returned by the transform function in the PyTorch dataloader is access here and is passed into the model.

In [ ]:
def train_model(loader, test_loader, model, epochs = 1):
    for epoch in range(epochs):  # loop over the dataset multiple times

        # Zero the performance stats for each epoch
        running_loss = 0.0
        start_time = time.time()
        total = 0
        correct = 0
        
        for i, data in enumerate(loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['images']
            labels = torch.squeeze(data['labels'])

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total
        
            # Print performance statistics
            running_loss += loss.item()
            if i % 10 == 0:    # print every 10 batches
                batch_time = time.time()
                speed = (i+1)/(batch_time-start_time)
                print('[%d, %5d] loss: %.3f, speed: %.2f, accuracy: %.2f %%' %
                      (epoch + 1, i, running_loss, speed, accuracy))

                running_loss = 0.0
        
        print('Testing Model Performance')
        test_model(test_loader, model)

    print('Finished Training')
    
    
def test_model(loader, model):
    start_time = time.time()
    total = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data['images']
            labels = torch.squeeze(data['labels'])

            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = 100 * correct / total
            
        print('Finished Testing')
        print('Testing accuracy: %.1f %%' %(accuracy))

The model and data are ready for training. Let's gooooooooooo 🚀!

In [ ]:
train_model(train_loader, test_loader, net, epochs = 1)

Congrats! You successfully trained an image classification model while streaming data directly from the cloud! 🎉